In [7]:
# !curl -L https://www.dropbox.com/s/rndzndlgpw3coiq/kaggle-download.zip -o kaggle-download.zip
# !unzip kaggle-download.zip -d kaggle_data


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17    0    17    0     0     58      0 --:--:-- --:--:-- --:--:--    58
100   340  100   340    0     0    401      0 --:--:-- --:--:-- --:--:--   401
100   534    0   534    0     0    408      0 --:--:--  0:00:01 --:--:--  2311
100  515M  100  515M    0     0  86.0M      0  0:00:05  0:00:05 --:--:--  121M
Archive:  kaggle-download.zip
replace kaggle_data/metaData_taxistandsID_name_GPSlocation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [1]:
import pandas as pd
from datetime import datetime
import torch


In [2]:
!pip install --upgrade torch torchvision

  Using cached torch-2.0.1-cp39-cp39-manylinux1_x86_64.whl (619.9 MB)
  Using cached torchvision-0.15.2-cp39-cp39-manylinux1_x86_64.whl (6.0 MB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached nvidia_nccl_cu11-2.14.3-py3-none-manylinux1_x86_64.whl (177.1 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached triton-2.0.0-1-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Using ca

# DATA STUFF

'A’ if this trip was dispatched from the central

‘B’ if this trip was demanded directly to a taxi driver on a specific stand

‘C’ otherwise (i.e. a trip demanded on a random street)

In [88]:
train = pd.read_csv('kaggle_data/train.csv')
train['ELAPSED'] = train['POLYLINE'].apply(lambda x: max(len(x.split('[')) - 2, 0) * 15)
train.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,ELAPSED
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",345
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",285
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",975
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",645
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",435


In [89]:
test = pd.read_csv('kaggle_data/test_public.csv')
test.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,T1,B,NaN,15.0,20000542,1408039037,A,False
1,T2,B,NaN,57.0,20000108,1408038611,A,False
2,T3,B,NaN,15.0,20000370,1408038568,A,False
3,T4,B,NaN,53.0,20000492,1408039090,A,False
4,T5,B,NaN,18.0,20000621,1408039177,A,False


In [90]:
call_A = train[train['CALL_TYPE'] == 'A']
call_B = train[train['CALL_TYPE'] == 'B']
call_C = train[train['CALL_TYPE'] == 'C']

call_C = call_C.sample(frac=.70) #.39

call_B = call_B.sample(frac=.40) #.38

call_A = call_A.sample(frac=.60) #.23


new_train = pd.concat([call_A, call_B, call_C], ignore_index=True)

train = new_train[(new_train['ELAPSED'] > 300) | (new_train['ELAPSED'] < 185)]

In [91]:
train.groupby('CALL_TYPE').count()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,ELAPSED
CALL_TYPE,,,,,,,,,
A,205650,205650,0,205650,205650,205650,205650,205650,205650
B,300614,0,296345,300614,300614,300614,300614,300614,300614
C,340717,0,0,340717,340717,340717,340717,340717,340717


If you have call type A, you have origin call
If you have call type B, you have origin stand
If you have call type C, you have neither

In [92]:
def parse_time(x):
  # We are using python's builtin datetime library
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 

train[["YR", "MON", "DAY", "HR", "WK"]] = train[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
train.head()

/tmp/ipykernel_114/1803232759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[["YR", "MON", "DAY", "HR", "WK"]] = train[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
/tmp/ipykernel_114/1803232759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[["YR", "MON", "DAY", "HR", "WK"]] = train[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
/tmp/ipykernel_114/1803232759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,ELAPSED,YR,MON,DAY,HR,WK
0,1398699006620000384,A,17486.0,NaN,20000384,1398699006,A,False,"[[-8.680392,41.161527],[-8.680878,41.161365],[...",945,2014,4,28,15,0
1,1399573780620000326,A,58241.0,NaN,20000326,1399573780,A,False,"[[-8.62173,41.16753],[-8.621757,41.167557],[-8...",675,2014,5,8,18,3
2,1388465848620000013,A,57461.0,NaN,20000013,1388465848,A,False,"[[-8.610318,41.185485],[-8.610327,41.185512],[...",1170,2013,12,31,4,1
3,1387384363620000344,A,6656.0,NaN,20000344,1387384363,A,False,"[[-8.611497,41.142231],[-8.611632,41.142591],[...",675,2013,12,18,16,2
4,1380035560620000005,A,56506.0,NaN,20000005,1380035560,A,False,"[[-8.624484,41.159961],[-8.624511,41.159961],[...",765,2013,9,24,15,1


In [93]:
train['WK'].value_counts()

4    137746
5    129955
3    121433
6    116223
1    115311
2    114071
0    112242
Name: WK, dtype: int64

In [94]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, drop='first')
encoded_data = encoder.fit_transform(train[['CALL_TYPE']])
column_names = encoder.get_feature_names(['CALL_TYPE'])
train_oh = pd.concat([train, pd.DataFrame(encoded_data, columns=column_names)], axis=1)

train_oh.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,ELAPSED,YR,MON,DAY,HR,WK,CALL_TYPE_B,CALL_TYPE_C
0,1.398699e+18,A,17486.0,NaN,20000384.0,1.398699e+09,A,False,"[[-8.680392,41.161527],[-8.680878,41.161365],[...",945.0,2014.0,4.0,28.0,15.0,0.0,0.0,0.0
1,1.399574e+18,A,58241.0,NaN,20000326.0,1.399574e+09,A,False,"[[-8.62173,41.16753],[-8.621757,41.167557],[-8...",675.0,2014.0,5.0,8.0,18.0,3.0,0.0,0.0
2,1.388466e+18,A,57461.0,NaN,20000013.0,1.388466e+09,A,False,"[[-8.610318,41.185485],[-8.610327,41.185512],[...",1170.0,2013.0,12.0,31.0,4.0,1.0,0.0,0.0
3,1.387384e+18,A,6656.0,NaN,20000344.0,1.387384e+09,A,False,"[[-8.611497,41.142231],[-8.611632,41.142591],[...",675.0,2013.0,12.0,18.0,16.0,2.0,0.0,0.0
4,1.380036e+18,A,56506.0,NaN,20000005.0,1.380036e+09,A,False,"[[-8.624484,41.159961],[-8.624511,41.159961],[...",765.0,2013.0,9.0,24.0,15.0,1.0,0.0,0.0


In [95]:
train_oh.groupby('ORIGIN_STAND')['ELAPSED'].mean().sort_values()

ORIGIN_STAND
62.0    470.334728
48.0    535.000000
8.0     608.684211
26.0    610.850216
28.0    625.569073
           ...    
45.0    812.209738
15.0    823.409393
39.0    826.096892
22.0    876.629630
59.0    913.571429
Name: ELAPSED, Length: 63, dtype: float64

In [96]:
# train_ftrs = train_oh[['ORIGIN_CALL', 'ORIGIN_STAND',  'CALL_TYPE_B', 'CALL_TYPE_C', 'YR', 'MON', 'DAY', 'HR', 'WK', 'ELAPSED']].fillna(-1)
train_ftrs = train_oh[['CALL_TYPE_B', 'CALL_TYPE_C', 'ORIGIN_STAND', 'MON', 'DAY', 'WK', 'HR', 'ELAPSED']].fillna(-1)

train_ftrs.head()

,CALL_TYPE_B,CALL_TYPE_C,ORIGIN_STAND,MON,DAY,WK,HR,ELAPSED
0,0.0,0.0,-1.0,4.0,28.0,0.0,15.0,945.0
1,0.0,0.0,-1.0,5.0,8.0,3.0,18.0,675.0
2,0.0,0.0,-1.0,12.0,31.0,1.0,4.0,1170.0
3,0.0,0.0,-1.0,12.0,18.0,2.0,16.0,675.0
4,0.0,0.0,-1.0,9.0,24.0,1.0,15.0,765.0


In [97]:
train_ftrs.dtypes

CALL_TYPE_B     float64
CALL_TYPE_C     float64
ORIGIN_STAND    float64
MON             float64
DAY             float64
WK              float64
HR              float64
ELAPSED         float64
dtype: object

# Model

In [98]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim


In [99]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
device

device(type='cuda')

In [100]:
from sklearn.model_selection import train_test_split

bs = 64

X = train_ftrs.drop(columns='ELAPSED')
y = train_ftrs['ELAPSED']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

num_ftrs = X_train.shape[1]

train_tn = torch.tensor(X_train.values, dtype=torch.float32).view(len(X_train), 1, num_ftrs).to(device)
exp_tn = torch.tensor(y_train.values, dtype=torch.float32).to(device)

val_tn = torch.tensor(X_val.values, dtype=torch.float32).view(len(X_val), 1, num_ftrs).to(device)
val_exp_tn = torch.tensor(y_val.values, dtype=torch.float32).to(device)

input_data = list(zip(train_tn, exp_tn))
input_data[0]

val_data = list(zip(val_tn, val_exp_tn))

trainloader = torch.utils.data.DataLoader(input_data, batch_size=bs)
valloader = torch.utils.data.DataLoader(val_data, batch_size=bs)

In [101]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)

        # Pass the input sequence through the RNN layer
        out, hidden = self.rnn(x, hidden)

        # Reshape the output to match the linear layer's input shape
        out = out.contiguous().view(-1, self.hidden_size)

        # Pass the output through the linear layer
        out = self.fc(out)

        return out

    def init_hidden(self, batch_size):
        # Initialize hidden state with zeros
        hidden = torch.zeros(1, batch_size, self.hidden_size).to(device)
        return hidden


In [102]:
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.2, reg_lambda=0.001):
        super(FNN, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.regularizer = nn.L1Loss()

        self.reg_lambda = reg_lambda

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x.transpose(1, 2)).transpose(1, 2)
        x = torch.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.bn2(x.transpose(1, 2)).transpose(1, 2)
        x = torch.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.bn3(x.transpose(1, 2)).transpose(1, 2)
        x = torch.relu(x)
        x = self.dropout(x)

        x = self.fc4(x)

        return x





In [103]:
# Define hyperparameters
input_size = num_ftrs  # number of features
hidden_size = input_size  # play around with and test 
output_size = 1  # dimension of output which is 1 elapsed time 

# model instatiation
# model = RNNModel(input_size, hidden_size, output_size).to(device)
model = FNN(input_size, hidden_size, output_size).to(device)

# Define the loss function and optimizer
mse = nn.MSELoss()
criterion = nn.SmoothL1Loss()

optimizer_adam = optim.Adam(model.parameters(), lr=0.0001)
# optimizer_sgd = optim.SGD(model.parameters(), lr=0.0001)

In [104]:
import tensorflow as tf
tf.test.gpu_device_name()

2023-06-08 22:14:39.048574: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /device:GPU:0 with 9627 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:89:00.0, compute capability: 6.1


'/device:GPU:0'

In [105]:
num_epochs = 15

# Training loop
for epoch in tqdm(range(num_epochs)):
    # Set the model in training mode
    model.train()
    for i, data in enumerate(trainloader):
        # print(i)
        inp, exp = data
    
        # Forward pass
        eps = .0000001
        outputs = model(inp)
        
        # Now doing RMSE
        # loss = torch.sqrt(criterion(outputs, exp) + eps)
        loss = torch.sqrt(mse(outputs, exp) + eps)

        # Backward pass and optimization
        optimizer_adam.zero_grad()
#         optimizer_sgd.zero_grad()
        loss.backward()
        optimizer_adam.step()
#         optimizer_sgd.step()
         
    # Validation loop
    model.eval()
    for i, data in enumerate(valloader):
        inp, exp = data
        with torch.no_grad():
            val_outputs = model(inp)
            val_loss = torch.sqrt(mse(val_outputs, exp) + eps)            
        
    # Print the loss for each epoch
    print(f"Epoch: {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}") 


  0%|          | 0/15 [00:00<?, ?it/s]/opt/conda/lib/python3.9/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.9/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([6, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.9/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([34])) that is different to the input size (torch.Size([34, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have

Epoch: 1, Training Loss: 622.7626953125, Validation Loss: 873.2701416015625


 13%|█▎        | 2/15 [01:34<10:19, 47.62s/it]

Epoch: 2, Training Loss: 584.6320190429688, Validation Loss: 844.1522216796875


 20%|██        | 3/15 [02:20<09:23, 46.94s/it]

...

Epoch: 13, Training Loss: 110.7107925415039, Validation Loss: 482.7375793457031


 93%|█████████▎| 14/15 [10:50<00:46, 46.24s/it]

Epoch: 14, Training Loss: 252.91795349121094, Validation Loss: 483.4246520996094


100%|██████████| 15/15 [11:33<00:00, 46.23s/it]

Epoch: 15, Training Loss: 235.8640594482422, Validation Loss: 483.498779296875


In [106]:
val_outputs.unique()

tensor([685.1354, 685.7918, 685.9456, 685.9549, 686.1978, 686.2012, 686.3281,
        686.3303, 686.3665, 686.3712, 686.3816], device='cuda:0')

In [ ]:
exp

In [ ]:
criterion = nn.MSELoss()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
t_plot = pd.DataFrame(t_loss)
x, y = t_plot.index, t_plot.values
a, b = np.polyfit(x, y, 1)
plt.scatter(x = x, y = y)
plt.plot(x, a*y+ b, color='red')

In [ ]:
v_plot = pd.DataFrame(v_loss)
x, y = v_plot.index, v_plot.values
a, b = np.polyfit(x, y, 1)
plt.scatter(x = x, y = y)
plt.plot(x, a*y+ b, color='red')

In [ ]:
t_ep_plot = pd.DataFrame(t_loss_ep)
x, y = t_ep_plot.index, t_ep_plot.values
a, b = np.polyfit(x, y, 1)
plt.scatter(x = x, y = y)
plt.plot(x, a*y+ b, color='red')

In [ ]:
v_ep_plot = pd.DataFrame(v_loss_ep)
x, y = v_plot.index, v_plot.values
a, b = np.polyfit(x, y, 1)
plt.scatter(x = x, y = y)
plt.plot(x, a*y+ b, color='red')

In [ ]:
model.eval()

In [ ]:
for i, data in enumerate(trainloader):
        # print(i)
        inp, exp = data
    
        # Forward pass
        eps = 1e-6
        outputs = model(inp)

In [ ]:
len(trainloader)


In [ ]:
outputs.unique()

In [ ]:
test

In [107]:
test[["YR", "MON", "DAY", "HR", "WK"]] = test[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

encoded_test_data = encoder.transform(test[['CALL_TYPE']])
test_oh = pd.concat([test, pd.DataFrame(encoded_test_data, columns=column_names)], axis=1)

test_ftrs = test_oh[['TRIP_ID', 'CALL_TYPE_B', 'CALL_TYPE_C', 'ORIGIN_STAND', 'MON', 'DAY', 'WK', 'HR']].fillna(-1)
trip_ids = test_ftrs['TRIP_ID']  # Save trip IDs for later


test_tn = torch.tensor(test_ftrs.drop(columns='TRIP_ID').values, dtype=torch.float32).view(len(test_ftrs), 1, num_ftrs).to(device)
testloader = torch.utils.data.DataLoader(test_tn, batch_size=16, shuffle=False)

model.eval()
predictions = []
for i, data in enumerate(testloader):
    with torch.no_grad():
        outputs = model(data)
    predictions.extend(outputs.squeeze(1).cpu().numpy())
    
for i in range(len(predictions)):
    predictions[i] = predictions[i][0]

submission = pd.DataFrame({'TRIP_ID': trip_ids, 'TRAVEL_TIME': predictions})
submission.to_csv('submission_ld.csv', index=False)


In [108]:
max(set(predictions))

686.5262

In [109]:
set(predictions)

{677.4716,
 677.7735,
 678.62054,
 684.8905,
 685.1354,
 685.32056,
 685.32715,
 685.3839,
 685.3875,
 685.5089,
 685.5347,
 685.5987,
 685.6524,
 685.65906,
 685.7005,
 685.71576,
 685.71936,
 685.743,
 685.746,
 685.7496,
 685.77313,
 685.7797,
 685.80634,
 685.8938,
 685.90375,
 685.95416,
 685.96075,
 686.02106,
 686.04767,
 686.0512,
 686.07733,
 686.108,
 686.11237,
 686.15204,
 686.1653,
 686.17194,
 686.2558,
 686.26245,
 686.2926,
 686.29333,
 686.31915,
 686.3463,
 686.34937,
 686.35254,
 686.35297,
 686.3765,
 686.383,
 686.3831,
 686.4067,
 686.41327,
 686.423,
 686.4368,
 686.453,
 686.50616,
 686.5108,
 686.5254,
 686.5262}